In [31]:
data = [
    "I love football",
    "The match was great",
    "I play basketball",
    "Basketball is fun to watch",
    "Coding in Python is fun",
    "I love programming",
    "Python is a great language"
]


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Tokenization, and TF-IDF representation
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(data)


In [33]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=0).fit(X)
clusters = kmeans.labels_


c:\Users\Thania\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [34]:
# Given labels based on our understanding
# 0 for football, 1 for basketball, 2 for coding
true_labels = [0, 0, 1, 1, 2, 2, 2]


In [36]:
import numpy as np
from sklearn.metrics import accuracy_score
from scipy.stats import mode

true_labels_np = np.array(true_labels)


labels = np.zeros_like(clusters)
for i in range(3):  # since we have 3 clusters
    mask = (clusters == i)
    mode_result = mode(true_labels_np[mask]).mode
    if np.isscalar(mode_result):  # If the mode result is a scalar
        labels[mask] = mode_result
    else:
        labels[mask] = mode_result[0]


accuracy = accuracy_score(true_labels_np, labels)
print(f"Accuracy: {accuracy * 100:.2f}%")



Accuracy: 71.43%
